In [36]:
from sentence_transformers import SentenceTransformer, util
import os
import time
import torch

if not torch.cuda.is_available():
    print("Warning: No GPU detected. Processing will be slow. Please add a GPU to this notebook")

model_name = 'quora-distilbert-multilingual'
model = SentenceTransformer(model_name)


# Get all unique sentences from the file
corpus_sentences = set()
import pandas as pd

corpus = pd.read_csv(r"C:\\Users\\maganti\\Documents\\INFO 5502 Chen Sir Project UNT\\Sentiment Analysis\\classification_question.txt", sep = "\n", header=None)
corpus[0] = corpus[0].apply(lambda x: " ".join(x.split()[1:]))


corpus_sentences = list((corpus[0]).values)
print("Encode the corpus. This might take a while")
corpus_embeddings = model.encode(corpus_sentences, show_progress_bar=True, convert_to_tensor=True)



###############################
print("Corpus loaded with {} sentences / embeddings".format(len(corpus_sentences)))

# Function that searches the corpus and prints the results
def search(inp_question):
    start_time = time.time()
    question_embedding = model.encode(inp_question, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, corpus_embeddings)
    end_time = time.time()
    #hits = hits[0]  #Get the hits for the first query
    
    #print("Input question:", inp_question)
    #print("Results (after {:.3f} seconds):".format(end_time-start_time))
    
    #for hit in hits[0]:
    #    print("\t{:.3f}\t{}".format(hit['score'], corpus_sentences[hit['corpus_id']]))

    return hits[0][0]
        
        
#result = search("So when you first started, can you describe a little bit about the office structure? Who was working? Who was in charge? How the whole office was set up?")


Encode the corpus. This might take a while


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Corpus loaded with 30 sentences / embeddings


In [37]:
search("Do you remember who it was that interviewed you?")

{'corpus_id': 4, 'score': 0.839056670665741}

In [ ]:
import json
import os
import glob
import pprint

classified_answers = {} 
count = 0
narrators_answers_corpus = ""
path = r'C:\\Users\\maganti\\Documents\\INFO 5502 Chen Sir Project UNT\\Web Scraping Interviews\\Narrators Json Files-With Q&A'
for filename in glob.glob(os.path.join(path, '*.json')): #only process .JSON files in folder.     
    #print(filename)
    with open(filename, mode = 'rb') as currentFile:
        data = json.loads(currentFile.read())
        try:
            if data['transcript'] != {}:
                for segment in data["transcript"].keys():
                    for question in data["transcript"][segment]['Segment Q&A'].keys():
                        #answers_sentiments[data["transcript"][segment]['Segment Q&A'][question]] = sentiment_scores(data["transcript"][segment]['Segment Q&A'][question])
                        #print(data["transcript"][segment]['Segment Q&A'][question])
                        result = search(question)
                        #print(result)
                        #print(data["transcript"][segment]['Segment Q&A'][question], result['corpus_id'])
                        try:
                            if result['corpus_id'] <= 20:
                                classified_answers[data["transcript"][segment]['Segment Q&A'][question]] = "1"

                            elif result['corpus_id'] == 21 or result['corpus_id'] == 22:
                                classified_answers[data["transcript"][segment]['Segment Q&A'][question]] = "2"

                            else:
                                classified_answers[data["transcript"][segment]['Segment Q&A'][question]] = "3"
                            count = count + 1
                        except Exception as e:
                            print("Exception is:",e)
                
            else:
                pass
        except Exception as e:
            print(e)
        
#print(len(segment_questions_list))
print()


In [50]:
search("And I understand that this was one of your first jobs after college. Could you talk a little bit about where you went to school and how you ended up in this role?")

{'corpus_id': 11, 'score': 0.9122568368911743}

In [69]:
df = pd.DataFrame(classified_answers.items())
df.columns = ['Answers', 'class']
df.to_csv("Classified_Answers.csv", index= False)

In [71]:
from transformers import pipeline
# function to print sentiments
# of the sentence.
count = 0
answers_sentiments = {}
classifier = pipeline("sentiment-analysis")

df['sentiment'] = df['Answers'].apply(lambda answer: classifier(answer, truncation=True)[0]['label'])

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


In [70]:
df['class'].value_counts()

1    88977
3    25142
2    14372
Name: class, dtype: int64

In [72]:
df.head()

,Answers,class,sentiment
0,"Well, my name is Aaron Zajic, and my role when...",3,NEGATIVE
1,"So I went to school at, it used to be called W...",1,NEGATIVE
2,Alicie West Simpson. I have completely lost tr...,1,NEGATIVE
3,So we were in a building on 1333 F Street whic...,1,NEGATIVE
4,"No, I started at DOL (Department of Labor).",1,NEGATIVE
